In [1]:
from simtk.openmm import app
import simtk.openmm as mm
from simtk import unit
from sys import stdout
import time as time

# what follows is from the OpenMM script builder
# with some minor modifications...

## Simulation setup

In [3]:
# read in a starting structure for ethane and the
# corresponding force field file
pdb = app.PDBFile('ethane.pdb')
forcefield = app.ForceField('ethane.gaff2.xml')

# setup system by taking topology from pdb file;
# run gas phase simulation with 2 fs time step (using SHAKE)
# at 298.15 K using a Langevin thermostat (integrator) with
# coupling constant of 5.0 ps^-1
system = forcefield.createSystem(pdb.topology, nonbondedMethod=app.NoCutoff, 
                                 constraints=app.HBonds)
integrator = mm.LangevinIntegrator(298.15*unit.kelvin, 5.0/unit.picoseconds, 
                                   2.0*unit.femtoseconds)
integrator.setConstraintTolerance(1e-5)

platform = mm.Platform.getPlatformByName('Reference')
simulation = app.Simulation(pdb.topology, system, integrator, platform)
simulation.context.setPositions(pdb.positions)

## Energy minimization

This reduces the potential energy of the system before beginning dynamics. This eliminates "bad" (i.e., overly close) contacts and generally leads to more stable simulation behavior.

In [4]:
print('Minimizing...')

st = simulation.context.getState(getPositions=True,getEnergy=True)
print("Potential energy before minimization is %s" % st.getPotentialEnergy())

simulation.minimizeEnergy(maxIterations=100)

st = simulation.context.getState(getPositions=True,getEnergy=True)
print("Potential energy after minimization is %s" % st.getPotentialEnergy())

Minimizing...
Potential energy before minimization is 4.467818224810632 kJ/mol
Potential energy after minimization is 4.390208155300384 kJ/mol


## Equilibration

We will run a (very) short equilibration simulation to bring the molecule up to our desired temperature.  If this were a periodic system, we would also aim to bring the density/volume to equilibrium at the desired pressure.

In [5]:
print('Equilibrating...')

simulation.reporters.append(app.StateDataReporter(stdout, 100, step=True, 
    potentialEnergy=True, temperature=True, separator=','))
simulation.context.setVelocitiesToTemperature(150.0*unit.kelvin)
simulation.step(2500)

Equilibrating...
#"Step","Potential Energy (kJ/mole)","Temperature (K)"
100,8.865043098692343,317.9472039540707
200,10.505824116117351,189.46036962689604
300,19.64392725647435,134.00599159054832
400,23.83005482847239,185.63894062224335
500,18.228328633463356,444.33900306402364
600,20.20804220847597,227.2016708268636
700,16.422818620500262,143.9946204742783
800,21.313265412928235,196.48446956419824
900,35.96869163049726,292.3946048400646
1000,24.904512756782047,378.0650760795068
1100,24.06577456588326,228.04617809458435
1200,32.94550810668696,178.07818910855917
1300,18.388438570624235,238.26057139546273
1400,21.960553380975604,237.17941528174197
1500,12.118535788398635,143.5554571326151
1600,16.554168274837934,357.8373189768155
1700,27.23767318404014,292.3270073491811
1800,12.84809274401041,150.57763074371954
1900,14.994203486158021,283.67653396043414
2000,18.121368091166573,226.18380954463825
2100,19.512950306356846,361.054269043839
2200,19.339072312570977,302.4907511371218
2300,13.343

## Production

Now we run a long MD simulation with parameters that are identical to the equilibration phase (other than simulation length, of course!).  We will also save a trajectory file (i.e., corodinats vs. time) of this simulation that we can analyze afterward using MDTraj (or other trajectory analysis tools).

In [6]:
print('Running Production...')

tinit=time.time()
simulation.reporters.clear()
# output basic simulation information below every 250000 steps/500 ps
simulation.reporters.append(app.StateDataReporter(stdout, 250000, 
    step=True, time=True, potentialEnergy=True, temperature=True, 
    speed=True, separator=','))
# write out a trajectory (i.e., coordinates vs. time) to a DCD
# file every 100 steps/0.2 ps
simulation.reporters.append(app.DCDReporter('ethane_sim.dcd', 100))

# run the simulation for 1.0x10^7 steps/20 ns
simulation.step(10000000)
tfinal=time.time()
print('Done!')
print('Time required for simulation:', tfinal-tinit, 'seconds')

Running Production...
#"Step","Time (ps)","Potential Energy (kJ/mole)","Temperature (K)","Speed (ns/day)"
250000,500.0000000016593,24.082052627841453,334.94512034837794,0
500000,999.9999999901769,25.344722414892193,234.91895323104566,2.15e+04
750000,1499.9999999783536,32.28107266330788,373.1073664372917,2.23e+04
1000000,1999.9999999665301,34.560289748553856,194.77691273286592,2.17e+04
1250000,2499.9999999547067,33.40893301951798,515.3034378942702,2.15e+04
1500000,2999.9999999428833,20.498903211479806,247.36017566284707,2.14e+04
1750000,3499.99999993106,25.184885811088407,514.463754752582,2.14e+04
2000000,3999.9999999192364,21.906141236767468,241.9436713736426,2.15e+04
2250000,4499.9999999992715,15.720297440738353,292.8362127679243,2.15e+04
2500000,5000.000000101135,22.837534145656008,266.72856430089047,2.15e+04
2750000,5500.000000202998,23.870937262426096,331.880423579939,2.15e+04
3000000,6000.000000304862,14.587143453109903,167.77322346389116,2.14e+04
3250000,6500.000000406725,10.6191